<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
data_path = 'data/ocv_features_normalized_241003.csv'

target_column_name = 'expt_ocv'
primary_column_name = 'original_md_ocv'
#primary_column_name = 'md_ocv'
using_primary_as_training = True

drop_col_list = ["cif_idx",'name']
#drop_col_list = ['SchNet']

self_define_train_test = True

random_seed = 461

#test_proportion = 0.1


In [2]:
# Parameters
file_options = "ocv_features_normalized_241003.csv"
proj_theme = "OCV"
drop_col_list = "cif_idx,name"
low_accuracy_column_name = "original_md_ocv"
high_accuracy_column_name = "expt_ocv"
random_seed = "461"


In [3]:
# Parameters
file_options = "ocv_features_normalized_241003.csv"
proj_theme = "OCV"
drop_col_list = "cif_idx,name"
low_accuracy_column_name = "original_md_ocv"
high_accuracy_column_name = "expt_ocv"
random_seed = "461"

In [4]:
data_path = '../uploads/' + file_options
target_column_name = high_accuracy_column_name
primary_column_name = low_accuracy_column_name
drop_col_list = drop_col_list.split(',')
random_seed = int(random_seed)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
from pprint import pprint

import sklearn.datasets
import sklearn.metrics 
from sklearn.model_selection import train_test_split

import autosklearn.regression
import matplotlib.pyplot as plt
import numpy as np

import utils
#importlib.reload(utils)
from utils import evaluation, plot_scatter
from sklearn.metrics import r2_score
import os

ModuleNotFoundError: No module named 'utils'

In [ ]:
save_dir_path = '../static/data/OCV/'

if not os.path.exists(save_dir_path):
    os.makedirs(save_dir_path)

In [ ]:
import pandas as pd
df_data = pd.read_csv(data_path,index_col=0)  
df_data.dropna(inplace=True,axis='columns')
df_data

In [ ]:
y = df_data[target_column_name]
X_all = df_data.drop(columns=[target_column_name])
X_all.drop(columns=drop_col_list,inplace=True)
X_no_primary = X_all.drop(columns=[primary_column_name])

X = X_all
if not using_primary_as_training:
    X = X_no_primary
    
# from sklearn.decomposition import PCA

# pca = PCA(n_components=3)

# # 拟合并转换数据
# X_pca = pca.fit_transform(X)
X

In [ ]:
df_data["name"].unique()

In [ ]:
if self_define_train_test:
    train_name, test_name = train_test_split(df_data["name"].unique(), test_size=0.2, random_state = random_seed)
    train_idx = df_data['name'].isin(train_name)
    test_idx = df_data['name'].isin(test_name)
    X_train = X[train_idx]
    X_test = X[test_idx]
    y_train = df_data[target_column_name][train_idx]
    y_test = df_data[target_column_name][test_idx]

else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_proportion, random_state=43)

if using_primary_as_training:
    name = [primary_column_name + ':'+ target_column_name,target_column_name,primary_column_name]
fig_0 = plot_scatter(y,X[primary_column_name],name=name)       
print('Overall')        
evaluation(X[primary_column_name],y)
print('\nTest')
evaluation(X_test[primary_column_name],y_test)

In [ ]:
test_name

In [ ]:
total_result = {}

time_left_for_this_task = 120
per_run_time_limit = 15

#for random_seed in range(460,500):
time_result = {}
train_name, test_name = train_test_split(df_data["name"].unique(), test_size=0.2, random_state = random_seed)
train_idx = df_data['name'].isin(train_name)
test_idx = df_data['name'].isin(test_name)
X_train = X[train_idx]
X_test = X[test_idx]
y_train = df_data[target_column_name][train_idx]
y_test = df_data[target_column_name][test_idx]

#for time_left_for_this_task in range(110,150,10):
automl = autosklearn.regression.AutoSklearnRegressor(
    memory_limit=8192,
    time_left_for_this_task = time_left_for_this_task,
    per_run_time_limit = per_run_time_limit,
)
automl.fit(X_train, y_train)

# print(automl.leaderboard())
# pprint(automl.show_models(), indent=4)

train_predictions = automl.predict(X_train)
test_predictions = automl.predict(X_test)
all_prediction = automl.predict(X)
df_data['prediction'] = all_prediction
#evaluation(test_predictions,y_test)
name = ['Auto-sklearn',target_column_name,'ML']

adc_fig = plot_scatter(y_train,train_predictions,y_test,test_predictions,name = name,label=['Train','Test'])

print('random seed:',random_seed,'\ntotal time:',time_left_for_this_task,'\nper run time:',per_run_time_limit)

#r2 = r2_score(y_test, test_predictions)

#time_result[time_left_for_this_task] = r2

#total_result[random_seed] = time_result



In [ ]:
# ocvplot：
test_fig_dic = {}
for moleculer in test_name:
    df_selected = df_data[df_data['name']==moleculer]

    test_fig = plt.figure()
    plt.title(moleculer)
    plt.grid(linewidth=1,linestyle='--')
    plt.plot(df_selected['Li_num'],df_selected['expt_ocv'],label='expt ocv',c='#000000')
    plt.plot(df_selected['Li_num'],df_selected['original_md_ocv'],label='original_md_ocv',c='#ecad9e')
    plt.plot(df_selected['Li_num'],df_selected['prediction'],label='ML',c='#19CAAD')
    plt.legend()

    test_fig_dic[moleculer] = test_fig


In [ ]:
# save the figure   

fig_0.savefig(save_dir_path+'0_before_delta.jpg')
adc_fig.savefig(save_dir_path+'2_adc.jpg')

for idx,moleculer in enumerate(test_name):
    test_fig_dic[moleculer].savefig(save_dir_path+'test_'+str(idx)+'.jpg')


In [ ]:
import yaml
import os

yaml_data = {
    "description": "OCV result analysis",
    "images": [
        f"/static/data/{proj_theme}/0_before_delta.jpg",
        f"/static/data/{proj_theme}/2_adc.jpg",]
        + [f"/static/data/{proj_theme}/test_{idx}.jpg" for idx in range(len(test_name))],
    "notes": [
        "Before Delta Plot. The x-axis is the original MD OCV value, and the y-axis is the experimental OCV value.",
        "Auto Delta Correction Plot",
    ]
}


with open(os.path.join(save_dir_path, f"config.yaml"), "w", encoding="utf-8") as f:
    yaml.dump(yaml_data, f, allow_unicode=True)
